#### BOW,TFIDF,Machine Learning Algorithms
(order very imp***)
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW And TF-IDF (Sentences--->vectors) {Preventing Data Leakage}
4. Trained Our Models


In [ ]:
import pandas as pd
messages=pd.read_csv('../smsspamcollection/SMSSpamCollection', sep='\t',names=["label","message"])

In [2]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
## Data Cleaning And Preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\salma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [5]:
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-z]',' ',messages['message'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [6]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

## Create Bag Of Words

In [7]:
## Output Features
y=pd.get_dummies(messages['label'])
y=y.iloc[:,0].values

In [8]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.20)

In [9]:
## Create the Bag OF Words model
from sklearn.feature_extraction.text import CountVectorizer
## for Binary BOW enable binary=True
cv=CountVectorizer(max_features=2500,ngram_range=(1,2))

In [10]:
len(X_train),len(y_train)

(4457, 4457)

In [11]:
## independent features
X_train=cv.fit_transform(X_train).toarray()
X_test=cv.transform(X_test).toarray()

In [12]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))
X_train

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0

In [13]:
cv.vocabulary_

{'cant': 295,
 'pick': 1584,
 'phone': 1578,
 'right': 1766,
 'pl': 1594,
 'send': 1844,
 'messag': 1314,
 'cant pick': 297,
 'pick phone': 1585,
 'phone right': 1581,
 'right pl': 1767,
 'pl send': 1601,
 'send messag': 1849,
 'sport': 1978,
 'type': 2227,
 'fantasi': 670,
 'back': 135,
 'tv': 2208,
 'go': 811,
 'sky': 1915,
 'activ': 15,
 'play': 1604,
 'dream': 573,
 'team': 2080,
 'score': 1819,
 'start': 1990,
 'saturday': 1807,
 'regist': 1733,
 'opt': 1522,
 'start saturday': 1992,
 'still': 2000,
 'around': 94,
 'could': 443,
 'use': 2284,
 'half': 908,
 'th': 2110,
 'cool': 433,
 'shall': 1873,
 'see': 1831,
 'anyway': 77,
 'home': 969,
 'got': 855,
 'someth': 1944,
 'drop': 578,
 'later': 1132,
 'let': 1169,
 'town': 2182,
 'tonight': 2173,
 'mayb': 1294,
 'mum': 1399,
 'take': 2065,
 'us': 2283,
 'happen': 915,
 'dear': 504,
 'silent': 1894,
 'free': 733,
 'msg': 1389,
 'bill': 175,
 'mobil': 1349,
 'number': 1478,
 'mistak': 1341,
 'pleas': 1608,
 'call': 250,
 'charg': 329

In [14]:
from sklearn.naive_bayes import MultinomialNB

In [15]:
spam_detect_model=MultinomialNB().fit(X_train,y_train)

In [16]:
y_pred=spam_detect_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [18]:
accuracy_score(y_test,y_pred)

0.9838565022421525

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.96      0.91      0.94       144
        True       0.99      0.99      0.99       971

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



### Creating The TF-IDF Model

In [20]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.20)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X_train=tv.fit_transform(X_train).toarray()
X_test=tv.transform(X_test).toarray()

In [22]:
X_train

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0

In [23]:
tv.vocabulary_

{'nobodi': 1445,
 'decid': 511,
 'eat': 597,
 'dad': 481,
 'want': 2330,
 'chines': 342,
 'wow': 2438,
 'never': 1425,
 'realiz': 1705,
 'thought': 2120,
 'like': 1160,
 'sinc': 1892,
 'best': 164,
 'could': 441,
 'alway': 55,
 'seem': 1829,
 'happi': 903,
 'sorri': 1941,
 'give': 791,
 'offer': 1480,
 'room': 1770,
 'bed': 156,
 'well': 2375,
 'amp': 58,
 'mom': 1354,
 'car': 293,
 'park': 1540,
 'long': 1191,
 'stori': 2008,
 'gonna': 825,
 'bore': 197,
 'day': 494,
 'internet': 1027,
 'sad': 1783,
 'one': 1502,
 'thing': 2108,
 'past': 1549,
 'good': 827,
 'morn': 1365,
 'good morn': 835,
 'leav': 1138,
 'come': 387,
 'home': 954,
 'ok': 1487,
 'make': 1245,
 'dinner': 545,
 'tonit': 2169,
 'invit': 1030,
 'lor': 1198,
 'thanx': 2102,
 'ok lor': 1490,
 'goin': 823,
 'nite': 1444,
 'aight': 38,
 'thank': 2098,
 'comin': 398,
 'way': 2348,
 'either': 604,
 'dude': 583,
 'knw': 1097,
 'also': 54,
 'tht': 2123,
 'gud': 880,
 'nyt': 1478,
 'gud nyt': 883,
 'dey': 533,
 'paid': 1534,
 'to

In [24]:
from sklearn.naive_bayes import MultinomialNB
spam_tfidf_model = MultinomialNB().fit(X_train, y_train)

In [28]:
#prediction
y_pred=spam_tfidf_model.predict(X_test)

In [26]:
score=accuracy_score(y_test,y_pred)
print(score)

0.9757847533632287


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       0.83      0.97      0.90       120
        True       1.00      0.98      0.99       995

    accuracy                           0.98      1115
   macro avg       0.91      0.98      0.94      1115
weighted avg       0.98      0.98      0.98      1115



In [34]:
test_text = ["win a billion dollars in one click"]
input = tv.transform(test_text).toarray()
result = spam_tfidf_model.predict(input)
print(result)
print("Spam" if result[0] == 1 else "Not Spam")

[ True]
Spam
